In [ ]:
import requests
import base64
import json
import time
import os, re
import string
from nltk import stem
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import wordpunct_tokenize
from gensim import corpora, models, similarities

def retrieve_readme(url1,suffix):
    r = requests.get(url1+'/readme')
    t = r.text
    dic = json.loads(t)
#    print(str(base64.b64decode(dic['content']).decode('UTF8')))
    with open('./Readme/readme'+suffix+'.md','a') as f:
        if 'content' in dic.keys():
            f.write(str(base64.b64decode(dic['content']).decode('UTF8')))

def data_retrieve():    
    search_condition = '?q=framework+language:javascript+stars:>1&sort=starts&order=dec'
    prefix = 'https://api.github.com/search/repositories'
    r1 = requests.get(prefix+search_condition)
    t1 = r1.text
    dict1 = json.loads(t1)
    print('number of retrieved repos:'+'\t'+str(dict1['total_count']))

    repos_number = dict1['total_count'] if dict1['total_count'] < 50 else 50
    i = 1
    for repo in dict1['items']:
        retrieve_readme(repo['url'],str(i))
        print(repo['url'])
        i = i + 1 
        time.sleep(1)
        if i > repos_number:
            break
            
def documents_iter(directory='./Readme'):
    for file in os.listdir(path=directory):
        if file[0:4]!='read':
            continue
        with open(directory+'/'+file,'r') as f:
            yield f
tw = ['\t','\n']
def stem_documents(documents_arr):
    lancaster = stem.lancaster.LancasterStemmer()
    def mystem (word):
            word = word.lower()
            if word not in tw:
                sword = lancaster.stem(word)
            else:
                sword = word
            return (sword)
    def mytokenize (doc):
        def mycleanfile_line(l):
            l = re.sub(r'http[\S]+[\s]',r' ',l)
            l = re.sub(r'[#\*=_]+',r' ',l)
            l = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_`{|}~]',r' ',l)
            l = re.sub(r'[\s]+',r' ',l)
            return l
        doc = ''.join([mycleanfile_line(line) for line in doc])
        return (word_tokenize(doc))
    texts = [[mystem(word) for word in mytokenize(document) if (word not in stopwords.words('english')) and (mystem(word) not in stopwords.words('english'))] for document in documents_arr()]
    #texts = [[mystem(word) for word in list(mytokenize(document)) if (word not in stopwords) and (mystem(word) not in stopwords)] for document in documents_arr()]
#all_tokens = sum (texts, []) 
#    tokens_once = set (word for word in set(all_tokens) if all_tokens.count(word) == 1)
#    texts = [[word for word in text if word not in tokens_once] for text in texts]
    #return texts
    return texts

    

def write_clean_data():
    with open('Readme_source.md','a') as f1:
        for file in stem_documents(documents_iter):
            for i in file:
                f1.write(i+' ')
            f1.write('\n')

model_word2vec = models.Word2Vec(stem_documents(documents_iter), min_count=10)
print(model_word2vec.vocab.keys())